In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random 
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
import PIL
import time
import cv2
import pandas as pd
from pylab import rcParams

In [ ]:
train_path = r"D:\Kenkyuu\Shin Dataset\train"
valid_path = r"D:\Kenkyuu\Shin Dataset\val"
test_path = r"D:\Kenkyuu\Shin Dataset\test"

In [ ]:
train_batches = ImageDataGenerator().flow_from_directory(
    directory=train_path, target_size=(224,224),classes = ['0', '1'], batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(
    directory=valid_path, target_size=(224,224),classes = ['0', '1'], batch_size=10)
test_batches = ImageDataGenerator().flow_from_directory(
    directory=test_path, target_size=(224,224),classes = ['0', '1'], batch_size=10, shuffle=False)

In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

In [ ]:
x = mobile.layers[-5].output
x = tf.keras.layers.Reshape(target_shape=(1024,))(x)
output = Dense(units=2, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=output)
model.summary()

In [ ]:
for layer in model.layers[:-23]:
    layer.trainable = False

model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

t1= time.time()
history = model.fit(x=train_batches,
                    steps_per_epoch=len(train_batches),
                    validation_data=valid_batches,
                    validation_steps=len(valid_batches),
                    epochs=30,
                    verbose=1
                    )
t2= time.time()
print('Time taken was {} seconds'.format(t2-t1))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss =history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy for softmax using categorical-crossentropy and Adam optimizer')
plt.grid()
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss for softmax using categorical-crossentropy and Adam optimizer')
plt.grid()
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
import PIL

dir_path = r"D:\Kenkyuu\Shin Dataset\test\1"

t3= time.time()

for i in os.listdir(dir_path):
    img = image.load_img(dir_path+'\\'+ i, target_size=(224,224))
    plt.imshow(img)
    plt.show()
    
    X = image.img_to_array(img)
    X = np.expand_dims(X,axis=0)
    images = np.vstack([X])
    val = model.predict(images)
    if val [0][0]> val[0][1]:
        print("not landing target")
        print(val)
    else:
        print("landing target")
        print(val)
        
t4= time.time()
print('Time taken was {} seconds'.format(t4-t3))

In [ ]:
from tensorflow.keras.preprocessing import image
import PIL

dir_path = r"D:\Kenkyuu\Shin Dataset\test\0"

t3= time.time()

for i in os.listdir(dir_path):
    img = image.load_img(dir_path+'\\'+ i, target_size=(224,224))
    plt.imshow(img)
    plt.show()
    
    X = image.img_to_array(img)
    X = np.expand_dims(X,axis=0)
    images = np.vstack([X])
    val = model.predict(images)
    if val [0][0]> val[0][1]:
        print("not landing target")
        print(val)
    else:
        print("landing target")
        print(val)
t4= time.time()
print('Time taken was {} seconds'.format(t4-t3))

In [ ]:
if os.path.isfile('models/mobilenet_landing_target_detection_model2.h5') is False:
    model.save('models/mobilenet_landing_target_detection_model2.h5')